In [62]:
import pickle 
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

True

In [57]:
REBUILD_DATA = True

class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open(r'I:\greyhound model\grvmodel\final.txt', 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    classes, inputs = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(inputs)):
            if len(classes[i]) == 8:
                self.training_data.append([np.array(inputs[i]),np.array(classes[i])])
            else:
                excluded += 1
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)
if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

excluded =  283


In [59]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

25104


In [67]:
Y

tensor([[1.0000, 0.2500, 0.1250,  ..., 0.3333, 0.1667, 0.2000],
        [0.2500, 0.3333, 0.5000,  ..., 0.1250, 0.1429, 0.1667],
        [0.1250, 0.1429, 0.1667,  ..., 0.3333, 0.5000, 1.0000],
        ...,
        [0.5000, 0.3333, 0.2500,  ..., 0.1429, 0.1250, 1.0000],
        [0.5000, 1.0000, 0.1250,  ..., 0.2000, 0.2500, 0.3333],
        [0.5000, 0.3333, 0.2500,  ..., 0.1250, 0.1667, 1.0000]])

In [60]:
X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

In [66]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(409, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
        
        
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=409, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=8, bias=True)
)


In [69]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 1

tensor([-7.6046,  1.7843, -5.4556,  3.6326,  4.7832, -1.6579,  1.1636, -4.7541],
       grad_fn=<AddBackward0>)

In [1]:
def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")
